In [1]:
import Pkg

import CairoMakie

using IntervalSets
using MMJMesh.Gmsh
using MMJMesh.Plots
using MMJMesh.Meshes
using MMJMesh.Utilities
using MMJMesh.Mathematics
using LinearAlgebra
using Symbolics
using DomainSets
using CairoMakie

using GLMakie
using WGLMakie
WGLMakie.activate!()

Pkg.add(url="https://github.com/matthiasbaitsch/mmjmesh.git")

    Updating git-repo `https://github.com/matthiasbaitsch/mmjmesh.git`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [2]:
include("fem.jl")

applyDirichletBCs! (generic function with 1 method)

### Netzgenerierung für rechteckige Platte

#### quadratische FEM-Elemente


In [3]:
l = 10
m = makemeshonrectangle(8.0, 8.0, l, l)
mplot(m,edgesvisible=true,edgecolor=:hotpink,nodesvisible=true, nodecolor=:blue, nodesize=5) |> mconf()

┌ Warning: Port in use, using different port. New port: 9391
└ @ Bonito.HTTPServer /Users/annapaganetty/.julia/packages/Bonito/5OnJB/src/HTTPServer/implementation.jl:278


In [4]:
p = (
    E = 31000e6,
    h = 0.2,
    ν = 0,
    q = 5e3);

In [5]:
m.data[:kefunc] = plateKe(p)
m.data[:refunc] = plateRe(p.q)

(::var"#reFunc#10"{Float64}) (generic function with 1 method)

In [6]:
K,r = assembleKr(m,3)
maximum(K\r)

1.1371343877676028e11

In [7]:
for rN ∈ m.groups[:boundarynodes]
    nodeD = nodeDOFs(rN,3)
    applyDirichletBCs!(nodeD, K, r)
end
ma = maximum(K\r)

0.001273039923892691

In [8]:
w_czerny = 5e3 * 8^4 / (31000e6 * 0.2^3) * 0.0152
100 * abs(ma - w_czerny) / w_czerny

1.4191962393951107